In [ ]:
from wordcloud import WordCloud, STOPWORDS
import plotly.graph_objs as go
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize

def freq_wordcloud(text):
    
    """
    This Function takes String as a input
    """

    def replace_verbs_and_plural_nouns(text):
        # Initialize the WordNet lemmatizer
        lemmatizer = WordNetLemmatizer()

        # Tokenize the text into words
        words = nltk.word_tokenize(text)

        # Tag each word with its part of speech
        pos_tags = nltk.pos_tag(words)

        # Replace verbs and plural nouns with their base form and singular form, respectively
        replaced_words = []
        for word, pos in pos_tags:
            if pos.startswith('V'):
                # Verb
                replaced_words.append(lemmatizer.lemmatize(word, pos='v'))
            elif pos == 'NNS':
                # Plural noun
                replaced_words.append(lemmatizer.lemmatize(word, pos='n'))
            else:
                # Other parts of speech
                replaced_words.append(word)

        # Reconstruct the text from the replaced words
        replaced_text = ' '.join(replaced_words)

        return replaced_text

    input_text = text
    input_text = replace_verbs_and_plural_nouns(input_text)
    porter = PorterStemmer()

    words = word_tokenize(input_text)
    stemmed_words = [porter.stem(word) for word in words]

    filtered_words = []
    for word in stemmed_words:
        if word.lower() not in stop_words:
            filtered_words.append(word)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    # Remove punctuations and convert to lowercase
    clean_text = re.sub(r'[^\w\s]', '', text).lower()

    # Split the text into words
    words = clean_text.split()

    # Create a dictionary to hold the word frequency
    word_freq = {}

    # Count the frequency of each word
    for word in filtered_words:
        if word in word_freq:
            word_freq[word] += 1
        else:
            word_freq[word] = 1
    # Create a DataFrame from the word frequency dictionary
    df = pd.DataFrame({'Word': list(word_freq.keys()), 'Frequency': list(word_freq.values())})

    # Sort the DataFrame in descending order of frequency
    df = df.sort_values(by=['Frequency','Word'], ascending=[False,True]).reset_index(drop=True)

    count_list = list(df['Frequency'].values)


    wc = WordCloud(stopwords = set(STOPWORDS),
                   max_words = 200,
                   max_font_size = 100)
    wc.generate(text)
    
    word_list=[]
    freq_list=[]
    fontsize_list=[]
    position_list=[]
    orientation_list=[]
    color_list=[]

    for (word, freq), fontsize, position, orientation, color in wc.layout_:
        word_list.append(word)
        freq_list.append(freq)
        fontsize_list.append(fontsize)
        position_list.append(position)
        orientation_list.append(orientation)
        color_list.append(color)
        
    # get the positions
    x=[]
    y=[]

    for i in position_list:
        x.append(i[0])
        y.append(i[1])
    sep =[]
    for i in range(len(x)):
        sep.append(':')        
    # get the relative occurence frequencies
    new_freq_list = []
    for i in freq_list:
        new_freq_list.append(i*100)
    new_freq_list
    
    trace = go.Scatter(x=x, 
                       y=y, 
                       textfont = dict(size=new_freq_list,
                                       color=color_list),
                       hoverinfo='text',
                
                       hovertext=['{0}{1}{2}'.format(w, s, f) for w, s, f in zip(word_list, sep,count_list)],
                       mode='text',  
                       text=word_list,
                      #  hovertemplate='<b>%{new_freq_list}</b><br>Frequency: %{marker.size}'
                      )
    
    layout = go.Layout({'xaxis': {'showgrid': False, 'showticklabels': False, 'zeroline': False},
                        'yaxis': {'showgrid': False, 'showticklabels': False, 'zeroline': False}})
    
    fig = go.Figure(data=[trace], layout=layout)
    
    return fig

